In [61]:
from sklearn.datasets import load_iris

In [62]:
data = load_iris()

In [63]:
data.keys()

dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename', 'data_module'])

In [64]:
data.data[0:5]

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2]])

In [65]:
data.target[0:5]

array([0, 0, 0, 0, 0])

In [66]:
target = data.target
data = data.data

In [67]:
bclass = {
    0 : [1,0,0],
    1 : [0,1,0],
    2 : [0,0,1]
}

In [68]:
y = []
for i in target:
    y.append(bclass[i])

y[0:5]

[[1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0]]

In [69]:
from sklearn.model_selection import train_test_split

train_data, test_data , train_target , test_target = \
    train_test_split(data,y,test_size=0.2 ,random_state=42)

In [70]:
train_data.shape

(120, 4)

In [71]:
test_data.shape

(30, 4)

In [72]:
train_data[0:5]

array([[4.6, 3.6, 1. , 0.2],
       [5.7, 4.4, 1.5, 0.4],
       [6.7, 3.1, 4.4, 1.4],
       [4.8, 3.4, 1.6, 0.2],
       [4.4, 3.2, 1.3, 0.2]])

In [73]:
train_target[0:5]

[[1, 0, 0], [1, 0, 0], [0, 1, 0], [1, 0, 0], [1, 0, 0]]

---
### Deep Learning

In [74]:
from tensorflow import keras

### Dense의 개수는 data * column + data 정도가 적당하다.

In [78]:
model = keras.Sequential()

model.add(keras.layers.Dense(600,activation='relu',input_shape=(4,)))
model.add(keras.layers.Dense(3,activation='softmax'))

In [79]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 600)               3000      
                                                                 
 dense_5 (Dense)             (None, 3)                 1803      
                                                                 
Total params: 4,803
Trainable params: 4,803
Non-trainable params: 0
_________________________________________________________________


In [80]:
import numpy as np 

model.compile(optimizer='adam',loss='categorical_crossentropy',metrics='accuracy')

model.fit(train_data,np.array(train_target),epochs=100)

Epoch 1/100
4/4 [==============================] - 0s 927us/step - loss: 1.0819 - accuracy: 0.3417
Epoch 2/100
4/4 [==============================] - 0s 861us/step - loss: 0.9458 - accuracy: 0.5417
Epoch 3/100
4/4 [==============================] - 0s 885us/step - loss: 0.8104 - accuracy: 0.7417
Epoch 4/100
4/4 [==============================] - 0s 2ms/step - loss: 0.7273 - accuracy: 0.9167
Epoch 5/100
4/4 [==============================] - 0s 1ms/step - loss: 0.6628 - accuracy: 0.9167
Epoch 6/100
4/4 [==============================] - 0s 2ms/step - loss: 0.6080 - accuracy: 0.7833
Epoch 7/100
4/4 [==============================] - 0s 681us/step - loss: 0.5648 - accuracy: 0.8333
Epoch 8/100
4/4 [==============================] - 0s 877us/step - loss: 0.5331 - accuracy: 0.9500
Epoch 9/100
4/4 [==============================] - 0s 1ms/step - loss: 0.5066 - accuracy: 0.9083
Epoch 10/100
4/4 [==============================] - 0s 984us/step - loss: 0.4851 - accuracy: 0.9750
Epoch 11/100
4/4 

---
### converting iris model to tflite

In [81]:
import tensorflow as tf

In [82]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
open('./Data/iris.tflite','wb').write(tflite_model)

INFO:tensorflow:Assets written to: /var/folders/vf/8xq5853s7zdffbbxdf_fk_xc0000gn/T/tmpx3x5v4g7/assets


INFO:tensorflow:Assets written to: /var/folders/vf/8xq5853s7zdffbbxdf_fk_xc0000gn/T/tmpx3x5v4g7/assets
2023-03-03 15:23:15.857080: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2023-03-03 15:23:15.857095: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2023-03-03 15:23:15.857864: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /var/folders/vf/8xq5853s7zdffbbxdf_fk_xc0000gn/T/tmpx3x5v4g7
2023-03-03 15:23:15.860933: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-03-03 15:23:15.860957: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /var/folders/vf/8xq5853s7zdffbbxdf_fk_xc0000gn/T/tmpx3x5v4g7
2023-03-03 15:23:15.866381: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:357] MLIR V1 optimization pass is not enabled
2023-03-03 15:23:15.867806: I tensorflow/cc/saved_model/load

20928

In [83]:
# h5 도 저장해서 크기 비교 해보자.
model.save('./Data/iris.h5')

In [86]:
# h5 모델을 tflite로
model = tf.keras.models.load_model('./Data/iris.h5')
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
open('./Data/iris2.tflite','wb').write(tflite_model)

INFO:tensorflow:Assets written to: /var/folders/vf/8xq5853s7zdffbbxdf_fk_xc0000gn/T/tmpp5vwerif/assets


INFO:tensorflow:Assets written to: /var/folders/vf/8xq5853s7zdffbbxdf_fk_xc0000gn/T/tmpp5vwerif/assets
2023-03-03 15:26:47.483309: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2023-03-03 15:26:47.483322: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2023-03-03 15:26:47.483409: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /var/folders/vf/8xq5853s7zdffbbxdf_fk_xc0000gn/T/tmpp5vwerif
2023-03-03 15:26:47.483878: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-03-03 15:26:47.483882: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /var/folders/vf/8xq5853s7zdffbbxdf_fk_xc0000gn/T/tmpp5vwerif
2023-03-03 15:26:47.485531: I tensorflow/cc/saved_model/loader.cc:229] Restoring SavedModel bundle.
2023-03-03 15:26:47.505300: I tensorflow/cc/saved_model/loader.cc:213] Running initialization

20928

In [90]:
# tflite 불러와서 예측하기
tflite_model = open('./Data/iris.tflite','rb').read()

# ... 알아보는 중